In [1]:
import polars as pl
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
def startTrain(X, y, criterion, splitter, max_depth, min_samples_split, min_samples_leaf):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        rus = RandomUnderSampler()
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)
        
        dt = DecisionTreeClassifier(
            criterion=criterion, splitter=splitter, max_depth=max_depth, 
            min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        
        start_training = time.time()
        dt.fit(X_train_resampled, y_train_resampled)
        end_training = time.time()

        y_pred = dt.predict(X_test)

        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        training_duration = end_training - start_training
        evaluation_duration = time.time() - end_training
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["DT"] + mean_results.tolist())

In [13]:
#Melhores Hiperparâmetros encontrados: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [14]:
for i in range(1,11):
    startTrain(X, y, criterion = 'entropy', splitter= 'best', max_depth = 10, min_samples_split = 2, min_samples_leaf = 1)
    print(i)

1
2
3
4
5
6
7
8
9
10


In [15]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""DT""",0.994962,0.996846,0.999927,0.994073,0.999619,0.996991,0.000381,17796.400391,6.8,552.599976,92690.796875,0.165598,0.10729
"""DT""",0.994923,0.996818,0.999924,0.994028,0.999607,0.996968,0.000393,17796.199219,7.0,556.799988,92686.601562,0.157305,0.090354
"""DT""",0.994896,0.99682,0.999933,0.993988,0.999652,0.996951,0.000348,17797.0,6.2,560.599976,92682.796875,0.150834,0.08822
"""DT""",0.994944,0.996867,0.999942,0.994037,0.999697,0.996981,0.000303,17797.800781,5.4,556.0,92687.398438,0.155236,0.095595
"""DT""",0.994894,0.996805,0.999927,0.993992,0.999618,0.99695,0.000382,17796.400391,6.8,560.200012,92683.203125,0.16416,0.090425
"""DT""",0.994925,0.996828,0.999929,0.994026,0.999629,0.996969,0.000371,17796.599609,6.6,557.0,92686.398438,0.161078,0.086185
"""DT""",0.994943,0.996852,0.999935,0.994041,0.999663,0.99698,0.000337,17797.199219,6.0,555.599976,92687.796875,0.153937,0.090432
"""DT""",0.994919,0.996834,0.999933,0.994015,0.999652,0.996966,0.000348,17797.0,6.2,558.0,92685.398438,0.162732,0.089766
"""DT""",0.994914,0.996817,0.999927,0.994015,0.999619,0.996962,0.000381,17796.400391,6.8,558.0,92685.398438,0.161969,0.088917


In [16]:
metrics_df.write_csv("metrics_results/balanced_dt_metrics_output.csv", separator=';')